In [15]:
!git clone https://github.com/shimming-toolbox/shimming-toolbox.git
%cd /content/shimming-toolbox/
!pip install shimming-toolbox/
! pip install plotly

In [2]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from skimage import io
import pandas as pd
import scipy as sp
import numpy as np
import os 


from shimmingtoolbox.unwrap.unwrap_phase import unwrap_phase
from shimmingtoolbox import __dir_testing__
from shimmingtoolbox.load_nifti import read_nii

# Introduction

In MRI, homogenous distribution of the main magnetic field ($B_0$) is essential to achieve high-quality images without having significant geometric distortion or undesirable signal dropout. Magnetic field inhomogeneity is expressed as parts per million of the magnetic field strength and is usually measured over a spherical volume within the magnet. While $B_0$ inhomogeneity can originate from hardware imperfections related to manufacturing or the presence of large metallic objects close to the magnet, the majority of field perturbations are subject-specific. The later field perturbations are the result of different magnetic susceptibility of tissues within the human body. Magnetic susceptibility ($\chi$) is a tissue-specific dimensionless parameter that describes the deviation of magnetic permeability of a given tissue (μ) with respect to the vacuum ($μ_0$), acccording to the following formula: 

$$ χ = (μ/μ_0) − 1$$

Apart from the magnitude of $\chi$ its sign is also important. Tissues with positive $\chi$ are referred to as paramagnetic and others with negative $\chi$ are called diamagnetic. Paramagnetic materials tend to enhance the $B_0$ by aligning towards it while diamagnetics weaken the $B_0$ by aligning in the opposite direction with respect to it.
Since most of the human body is consist of water as a diamagnetic ($\chi = −9.05 × 10^{−6} = −9.05  ppm $) material most of the tissues inside the body are therefore diamagnetic. On the other hand, the most commonly encountered paramagnetic material is air ($\chi = +0.36 × 10^{−6} = 0.36  ppm $). 

Taking the brain as an example, the most intense $B_0$ variations are corresponding to the skull base where there are ear canal and air sinuses along with different brain tissues. These local susceptibility differences alter the Larmor frequency ($\omega $) leading to phase accumulation over time. The greater the difference of $\chi$ in a volume of interest the greater the spread of phases at the echo time.

$$\omega_0 = \gamma B_0$$

Thus, the reconstructed phase map from the complex signal of MRI contains information related to $B_0$ field variation. However, apart from susceptibility-induced inhomogeneity, there are other contributing factors in phase maps like errors related to the centering of the sampling window, Rf penetration issues especially at high field strength which vary by distance from the surface of the object being imaged as well as problems with Rf receiver that lead to phase variation. 
Due to the involvement of different sources in phase variations, phase maps can only approximate an upper level of the $\Delta B_0$ whereas the real variation would be lower. As most of these sources will not change with changing echo time except to the $B_0$ inhomogeneity-induced phase that scales with echo time. So to achieve a $\Delta B_0$ field map at least two phase maps with different echoes must be acquired. 

  
Hardware-related inhomogeneities are often accounted for by means of passive shimming meaning inserting metallic sheets (usually steel) to compensate the local field inhomogenity. However, this approach fails to provide a robust shimming solution when it comes to considering inter-subject variability due to anatomical discrepancies as well as positioning differences and angulation with respect to the $B_0$ direction (s. Clare et al. 2006). 

The subject-specific magnetic field distribution is often obtained from the phase evolution during a period of time using an appropriate MRI sequence. While MRI-based $B_0$ mapping can essentially be based on any MRI pulse sequence, the fast gradient-echo (gre) method is commonly used owing to its speed, ease of use, and inherent sensitivity to $B_0$ offsets. 

There are two general methods to calculate the $B_0$ field map namely: phase subtraction and linear fitting approaches.

The problem with phase images is that everything beyond the frequnecy of $\frac {1} {\Delta TE}$ will wrapp into $2\pi$ range.

Usually the echo time for the first phase data and the $\Delta TE $ for the linear fitting approuch are chose to be low enough to prevent wrapping.

In [ ]:
interleaved_phase = read_nii('/Users/behrouz/Documents/Code/pythonProject/interleaved_echo_phases.nii')[0]
mask = read_nii('/Users/behrouz/Documents/Code/pythonProject/interleaved_echoes_mag_brain_mask.nii')[0]
data = interleaved_phase.get_fdata()
mask = mask.get_fdata()
echotimes = [2.6, 5.2, 9, 13]
slice_1 = data[..., 176:180] * mask[..., 176:180]
slice_2 = data[..., 220:224] * mask[..., 224:228]

pro_RL1 = [slice_1[:,27,i] for i in range(slice_1.shape[2])]
pro_AP1 = [slice_1[27,:,i] for i in range(slice_1.shape[2])]
pro_RL2 = [slice_2[:,27,i] for i in range(slice_1.shape[2])]
pro_AP2 = [slice_2[27,:,i] for i in range(slice_1.shape[2])]
xAxis  = slice_1.shape[0]
yAxis = slice_1.shape[1]


img = make_subplots(rows=3, cols=2, 
                    column_titles= ['Lower slice','Upper slice'], row_heights=[0.6, 0.2, 0.2])

for step in np.arange(0, 4, 1):
    img.add_trace(go.Heatmap(z = slice_1[...,step].T,
                             x = list(range(0, xAxis)),
                             y = list(range(0, yAxis)),
                             colorscale='Greys', name = 'phaseVal', visible=False),
                             row=1, col=1)
    img.add_trace(go.Heatmap(z = slice_2[...,step].T,
                             x = list(range(0, xAxis)),
                             y = list(range(0, yAxis)),
                             colorscale='Greys', name = 'phaseVal', visible=False),
                             row=1, col=2)
    img.add_scatter(y = np.flip(pro_AP1[step]), row=2 , col=1, visible= False, name = 'phaseVal')
    img.add_scatter(y = np.flip(pro_AP2[step]), row=2 , col=2, visible= False, name = 'phaseVal')
    img.add_scatter(y = pro_RL1[step], row=3 , col=1, visible= False, name = 'phaseVal')
    img.add_scatter(y = pro_RL2[step], row=3 , col=2, visible= False, name = 'phaseVal')


img.data[0]['visible'] = True
img.data[1]['visible'] = True
img.data[2]['visible'] = True
img.data[3]['visible'] = True
img.data[4]['visible'] = True
img.data[5]['visible'] = True

steps = []
for i, j in zip(range(0, len(img.data), 6), range(4)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(img.data)], label = str(echotimes[j]))
    step['args'][1][i] = True
    step['args'][1][i+1] = True
    step['args'][1][i+2] = True
    step['args'][1][i+3] = True
    step['args'][1][i+4] = True
    step['args'][1][i+5] = True
    steps.append(step)

sliders = [dict(
    active =0,
    currentvalue = {"prefix": "Echo time: ", "suffix":" ms" },
    steps = steps)]

img.layout.sliders = sliders
img.update_yaxes(title_text="Phase map", row=1, col=1)
img.update_yaxes(title_text="AP projection", row=2, col=1)
img.update_yaxes(title_text="RL projection", row=3, col=1)
img.update_layout(height=800, width=800, showlegend=False, template='plotly_white', )


go.FigureWidget(img)
# img.write_html("file.html")


FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(255,255,255)'], [0.125,
                             'r…

## Phase subtraction field mapping

The field offset can be calculated based on phase difference between the two images with different echo times as following.
$$\Delta B_0 = \frac{Δφ}{\Delta TE.2π} $$

## Linear fitting field mapping